## Data Preparation
The dataset contains around 2500 images of bees and around 2100 images of wasps.

The dataset contains separate folders for training and test sets.

Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

The shape for input should be (150, 150, 3)
Next, create a convolutional layer (Conv2D):
Use 32 filters
Kernel size should be (3, 3) (that's the size of the filter)
Use 'relu' as activation
Reduce the size of the feature map with max pooling (MaxPooling2D)
Set the pooling size to (2, 2)
Turn the multi-dimensional result into vectors using a Flatten layer
Next, add a Dense layer with 64 neurons and 'relu' activation
Finally, create the Dense layer with 1 neuron - this will be the output
The output layer should have an activation - use the appropriate activation for the binary classification case
As optimizer use SGD with the following parameters:

SGD(lr=0.002, momentum=0.8)


In [6]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.xception import preprocess_input
# from keras.applications import Xception
# from keras import layers
# from keras import models
# from keras import optimizers

In [ ]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
# !unzip data.zip

In [2]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(150, 150),
    batch_size=32
)

Found 3677 images belonging to 2 classes.


In [3]:
train_ds.class_indices

{'bee': 0, 'wasp': 1}

In [4]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_ds = val_gen.flow_from_directory(
    './data/test',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

Found 918 images belonging to 2 classes.


In [10]:
#size for the inner dense layer
size_inner = 64

#the learning rate
learning_rate = 0.002



base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

base_model.trainable = False

    #########################################

inputs = keras.Input(shape=(150, 150, 3))
base = base_model(inputs, training=False)

#vectors = keras.layers.GlobalAveragePooling2D()(base)
conv_layer = layers.Conv2D(32, (3, 3), activation='relu')(base)
pooling_layer = layers.MaxPooling2D((2, 2))(conv_layer)
flattened = layers.Flatten()(pooling_layer)
inner = layers.Dense(units=size_inner, activation='relu')(flattened) 
#inner = keras.layers.Dense(size_inner=64, activation='relu')(flattened) 
#outputs = keras.layers.Dense(10)(inner)
outputs = layers.Dense(1, activation='sigmoid')(inner)
#model = keras.Model(inputs, outputs)
model = models.Model(inputs, outputs)
    
    #########################################

#optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = optimizers.SGD(learning_rate=0.002, momentum=0.8)
#loss = keras.losses.CategoricalCrossentropy(from_logits=True)
loss = 'binary_crossentropy'

model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy'])

## Question 1
Since we have a binary classification problem, what is the best loss function for us?

In [ ]:
# binary_crossentropy

## Question 2
What's the number of parameters in the convolutional layer of our model? You can use the summary method for that.

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 conv2d_23 (Conv2D)          (None, 3, 3, 32)          589856    
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 1, 1, 32)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                           

In [12]:
model.summary()
conv_layer_params = model.layers[2].count_params()  #the Conv2D layer is the third layer
print("Number of parameters in the convolutional layer:", conv_layer_params)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 conv2d_23 (Conv2D)          (None, 3, 3, 32)          589856    
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 1, 1, 32)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                           

## Generators and Training
For the next two questions, use the following data generator for both train and test sets:

ImageDataGenerator(rescale=1./255)
We don't need to do any additional pre-processing for the images.
When reading the data from train/test directories, check the class_mode parameter. Which value should it be for a binary classification problem?
Use batch_size=20
Use shuffle=True for both training and test sets.
For training use .fit() with the following params:

model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
Question 3
What is the median of training accuracy for all the epochs for this model?

In [21]:
train_gen = ImageDataGenerator(
                               rescale=1./255)
train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

val_gen = ImageDataGenerator(
                             rescale=1./255)
val_ds = val_gen.flow_from_directory(
    './data/test',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [15]:
train_ds.class_mode

'categorical'

In [16]:
val_ds.class_mode

'categorical'

In [22]:
#size for the inner dense layer
size_inner = 64

#the learning rate
learning_rate = 0.002



base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

base_model.trainable = False

    #########################################

inputs = keras.Input(shape=(150, 150, 3))
base = base_model(inputs, training=False)

#vectors = keras.layers.GlobalAveragePooling2D()(base)
conv_layer = layers.Conv2D(32, (3, 3), activation='relu')(base)
pooling_layer = layers.MaxPooling2D((2, 2))(conv_layer)
flattened = layers.Flatten()(pooling_layer)
inner = layers.Dense(units=size_inner, activation='relu')(flattened) 
#inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
#outputs = keras.layers.Dense(10)(inner)
outputs = layers.Dense(1, activation='sigmoid')(inner)
#model = keras.Model(inputs, outputs)
model = models.Model(inputs, outputs)
    
    #########################################

#optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = optimizers.SGD(learning_rate=0.002, momentum=0.8)
#loss = keras.losses.CategoricalCrossentropy(from_logits=True)
loss = 'binary_crossentropy'

model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy'])

history=model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
184/184 [==============================] - 82s 430ms/step - loss: 0.3123 - accuracy: 0.8648 - val_loss: 0.2128 - val_accuracy: 0.9205
Epoch 2/10
184/184 [==============================] - 78s 427ms/step - loss: 0.2296 - accuracy: 0.9111 - val_loss: 0.2152 - val_accuracy: 0.9150
Epoch 3/10
184/184 [==============================] - 79s 431ms/step - loss: 0.1925 - accuracy: 0.9244 - val_loss: 0.2279 - val_accuracy: 0.9129
Epoch 4/10
184/184 [==============================] - 79s 429ms/step - loss: 0.1640 - accuracy: 0.9372 - val_loss: 0.1990 - val_accuracy: 0.9194
Epoch 5/10
184/184 [==============================] - 79s 430ms/step - loss: 0.1451 - accuracy: 0.9448 - val_loss: 0.2066 - val_accuracy: 0.9194
Epoch 6/10
184/184 [==============================] - 79s 430ms/step - loss: 0.1226 - accuracy: 0.9546 - val_loss: 0.2218 - val_accuracy: 0.9139
Epoch 7/10
184/184 [==============================] - 79s 428ms/step - loss: 0.0989 - accuracy: 0.9649 - val_loss: 0.2396 - val_ac

In [23]:
training_accuracy = history.history['accuracy']
training_loss = history.history['loss']


median_training_accuracy = np.median(training_accuracy)
print("Median of training accuracy:", median_training_accuracy)

Median of training accuracy: 0.9496872425079346


## Question 4
What is the standard deviation of training loss for all the epochs for this model?

In [24]:
# Calculate the standard deviation of training loss
std_training_loss = np.std(training_loss)
print("Standard deviation of training loss:", std_training_loss)


Standard deviation of training loss: 0.07742216594427309


## Data Augmentation
For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

rotation_range=50,
width_shift_range=0.1,
height_shift_range=0.1,
zoom_range=0.1,
horizontal_flip=True,
fill_mode='nearest'
Question 5
Let's train our model for 10 more epochs using the same code as previously.

Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

In [26]:
train_gen = ImageDataGenerator(rescale=1./255,
                              rotation_range=50,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                fill_mode='nearest')
train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

val_gen = ImageDataGenerator(
                             rescale=1./255)
val_ds = val_gen.flow_from_directory(
    './data/test',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [27]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
184/184 [==============================] - 96s 522ms/step - loss: 0.2869 - accuracy: 0.8809 - val_loss: 0.2142 - val_accuracy: 0.9139
Epoch 2/10
184/184 [==============================] - 92s 498ms/step - loss: 0.2465 - accuracy: 0.8999 - val_loss: 0.2244 - val_accuracy: 0.9237
Epoch 3/10
184/184 [==============================] - 86s 470ms/step - loss: 0.2342 - accuracy: 0.9070 - val_loss: 0.2331 - val_accuracy: 0.9227
Epoch 4/10
184/184 [==============================] - 86s 468ms/step - loss: 0.2278 - accuracy: 0.9064 - val_loss: 0.2067 - val_accuracy: 0.9248
Epoch 5/10
184/184 [==============================] - 85s 463ms/step - loss: 0.2286 - accuracy: 0.9083 - val_loss: 0.2070 - val_accuracy: 0.9248
Epoch 6/10
184/184 [==============================] - 85s 461ms/step - loss: 0.2109 - accuracy: 0.9151 - val_loss: 0.2195 - val_accuracy: 0.9248
Epoch 7/10
184/184 [==============================] - 85s 463ms/step - loss: 0.2074 - accuracy: 0.9108 - val_loss: 0.2206 - val_ac

In [28]:
# Extract test loss and accuracy from the history object
test_loss = history.history['val_loss']
test_accuracy = history.history['val_accuracy']

# Calculate the mean of test loss for all epochs
mean_test_loss= np.mean(test_loss)
print("Mean of test loss for all epochs:", mean_test_loss)



Mean of test loss for all epochs: 0.21940127164125442


## Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [29]:
# Calculate the average of test accuracy for the last 5 epochs
average_test_accuracy_last5 = np.mean(test_accuracy[-5:])
print("Average of test accuracy for the last 5 epochs:", average_test_accuracy_last5)

Average of test accuracy for the last 5 epochs: 0.9248365998268128
